>v0.1 This code implements a simple feature extraction and train using Lightgbm.

Feature extraction is very simple and can be improved.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import librosa
import matplotlib.pyplot as plt
import gc

from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.metrics import roc_auc_score

from scipy import stats

from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

In [2]:
def split_and_label(rows_labels):
    
    row_labels_list = []
    for row in rows_labels:
        row_labels = row.split(',')
        labels_array = np.zeros((80))
        
        for label in row_labels:
            index = label_mapping[label]
            labels_array[index] = 1
        
        row_labels_list.append(labels_array)
    
    return row_labels_list

In [3]:
def create_features( pathname ):

    y, sr = librosa.load( pathname)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y)
    xc = pd.Series(y)
    
    X = []
    X.append(len(xc)/sr)
    X.append( xc.mean() )
    X.append( xc.median() )
    X.append( xc.std() )
    X.append( xc.max() )
    X.append( xc.min() )
    X.append( xc.skew() )
    X.append( xc.mad() )
    X.append( xc.kurtosis() )
    
    X.append( np.mean(np.diff(xc)) )
    X.append( np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0]) )
    X.append( np.abs(xc).max() )
    X.append( np.abs(xc).min() )
    
    X.append( xc[:4410].std() )
    X.append( xc[-4410:].std() )
    X.append( xc[:44100].std() )
    X.append( xc[-44100:].std() )
    
    X.append( xc[:4410].mean() )
    X.append( xc[-4410:].mean() )
    X.append( xc[:44100].mean() )
    X.append( xc[-44100:].mean() )
    
    X.append( xc[:4410].min() )
    X.append( xc[-4410:].min() )
    X.append( xc[:44100].min() )
    X.append( xc[-44100:].min() )
    
    X.append( xc[:4410].max() )
    X.append( xc[-4410:].max() )
    X.append( xc[:44100].max() )
    X.append( xc[-44100:].max() )
    
    X.append( xc[:4410].skew() )
    X.append( xc[-4410:].skew() )
    X.append( xc[:44100].skew() )
    X.append( xc[-44100:].skew() )
    
    X.append( xc.max() / np.abs(xc.min()) )
    X.append( xc.max() - np.abs(xc.min()) )
    X.append( xc.sum() )
    
    X.append( np.mean(np.nonzero((np.diff(xc[:4410]) / xc[:4410][:-1]))[0]) )
    X.append( np.mean(np.nonzero((np.diff(xc[-4410:]) / xc[-4410:][:-1]))[0]) )
    X.append( np.mean(np.nonzero((np.diff(xc[:44100]) / xc[:44100][:-1]))[0]) )
    X.append( np.mean(np.nonzero((np.diff(xc[-44100:]) / xc[-44100:][:-1]))[0]) )
    
    X.append( np.quantile(xc, 0.95) )
    X.append( np.quantile(xc, 0.99) )
    X.append( np.quantile(xc, 0.10) )
    X.append( np.quantile(xc, 0.05) )
    
    X.append( np.abs(xc).mean() )
    X.append( np.abs(xc).std() )
             
    return np.array( X )

In [26]:
train_curated = pd.read_csv('../data/raw/train_curated.csv')
#train_noisy_labels = pd.read_csv('../data/raw/train')
test = pd.read_csv('../data/raw/sample_submission.csv')

print(train_curated.shape, test.shape)

(4970, 2) (1120, 81)


In [5]:
label_columns = list( test.columns[1:] )
label_mapping = dict((label, index) for index, label in enumerate(label_columns))
label_mapping

{'Accelerating_and_revving_and_vroom': 0,
 'Accordion': 1,
 'Acoustic_guitar': 2,
 'Applause': 3,
 'Bark': 4,
 'Bass_drum': 5,
 'Bass_guitar': 6,
 'Bathtub_(filling_or_washing)': 7,
 'Bicycle_bell': 8,
 'Burping_and_eructation': 9,
 'Bus': 10,
 'Buzz': 11,
 'Car_passing_by': 12,
 'Cheering': 13,
 'Chewing_and_mastication': 14,
 'Child_speech_and_kid_speaking': 15,
 'Chink_and_clink': 16,
 'Chirp_and_tweet': 17,
 'Church_bell': 18,
 'Clapping': 19,
 'Computer_keyboard': 20,
 'Crackle': 21,
 'Cricket': 22,
 'Crowd': 23,
 'Cupboard_open_or_close': 24,
 'Cutlery_and_silverware': 25,
 'Dishes_and_pots_and_pans': 26,
 'Drawer_open_or_close': 27,
 'Drip': 28,
 'Electric_guitar': 29,
 'Fart': 30,
 'Female_singing': 31,
 'Female_speech_and_woman_speaking': 32,
 'Fill_(with_liquid)': 33,
 'Finger_snapping': 34,
 'Frying_(food)': 35,
 'Gasp': 36,
 'Glockenspiel': 37,
 'Gong': 38,
 'Gurgling': 39,
 'Harmonica': 40,
 'Hi-hat': 41,
 'Hiss': 42,
 'Keys_jangling': 43,
 'Knock': 44,
 'Male_singing': 45

In [6]:
len(label_mapping)

80

In [7]:
train_curated_labels = split_and_label(train_curated['labels'])
#train_noisy_labels   = split_and_label(train_noisy  ['labels'])
len(train_curated_labels)#, len(train_noisy_labels)

4970

In [8]:
for f in label_columns:
    train_curated[f] = 0.0
    #train_noisy[f] = 0.0

In [9]:
train_curated[label_columns] = train_curated_labels
#train_noisy[label_columns]   = train_noisy_labels

train_curated['num_labels'] = train_curated[label_columns].sum(axis=1)
#train_noisy['num_labels']   = train_noisy[label_columns].sum(axis=1)

train_curated['path'] = '../data/raw/train_curated/'+train_curated['fname']
#train_noisy  ['path'] = '../input/train_noisy/'+train_noisy['fname']

train_curated.head()

,fname,labels,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),...,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing),num_labels,path
0,0006ae4e.wav,Bark,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../data/raw/train_curated/0006ae4e.wav
1,0019ef41.wav,Raindrop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../data/raw/train_curated/0019ef41.wav
2,001ec0ad.wav,Finger_snapping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../data/raw/train_curated/001ec0ad.wav
3,0026c7cb.wav,Run,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../data/raw/train_curated/0026c7cb.wav
4,0026f116.wav,Finger_snapping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../data/raw/train_curated/0026f116.wav


In [10]:
test['path'] = '../data/raw/test/' + test['fname']


In [11]:
train = train_curated
del train_curated
gc.collect()
train.shape

(4970, 84)

#### Making features from train curated

In [12]:

X = [create_features(fn) for fn in tqdm(train['path'].values)] 


100%|██████████| 4970/4970 [19:56<00:00,  2.71it/s]


In [13]:
X = np.array(X)

In [14]:
X.shape

(4970, 46)

In [15]:
np.save('../data/processed/train_curated_features.npy', X)

#### Making features from test curated

In [12]:
X = [create_features(fn) for fn in tqdm(test['path'].values)] 

100%|██████████| 1120/1120 [05:27<00:00,  3.33it/s]


In [13]:
X = np.array(X)

In [14]:
X.shape

(1120, 46)

In [15]:
np.save('../data/processed/test_features.npy', X)

In [ ]:
train_curated.index